In [1]:
#for u in GF(q), we can factor as u=aa^* using gen. z and modular arithmetic
def conj_square_root(u):
    if u == 0:
        return 0  # Special case for 0
    z = F.multiplicative_generator()
    k = u.log(z)  # Compute discrete log of u to the base z
    if k % (q+1) != 0:
        raise ValueError("Unable to factor: u is not in base field GF(q)")
    return z**(k//(q+1))

In [2]:
def base_change_hermitian_gap_source(mat):
    """
    Diagonalizes a Hermitian matrix over a finite field.
    Returns the base change matrix and the rank of the Hermitian form.

    Arguments:
        mat: The Gram matrix of a Hermitian form (Sage matrix object)

    Returns:
        D: The base change matrix
        r: The number of non-zero rows in D*mat*D^T
    """
    assert mat.nrows() == mat.ncols()
    
    F = mat.base_ring()
    n = mat.nrows()
    q = sqrt(F.order())

    if n == 1:
        return matrix(F, [conj_square_root(mat[0][0])])

    A = copy(mat)
    D = identity_matrix(F, n)
    row = -1  # Start at -1 to align with zero-based indexing

    while True:
        row += 1  # Increment at the start of each loop

        # Look for a non-zero element on the main diagonal, starting from `row`
        i = row
        while i < n and A[i, i].is_zero():
            i += 1

        if i == row:
            # Do nothing since A[row, row] != 0
            pass
        elif i < n:
            # Swap to ensure A[row, row] != 0
            A.swap_rows(row, i)
            A.swap_columns(row, i)
            D.swap_rows(row, i)
        else:
            # All entries on the main diagonal are zero; look for an off-diagonal element
            i = row
            while i < n - 1:
                k = i + 1
                while k < n and A[i, k].is_zero():
                    k += 1
                if k == n:
                    i += 1
                else:
                    break

            if i == n - 1:
                # All elements are zero; terminate
                row -= 1
                r = row + 1
                break

            # Fetch the non-zero element and place it at A[row, row + 1]
            if i != row:
                A.swap_rows(row, i)
                A.swap_columns(row, i)
                D.swap_rows(row, i)

            A.swap_rows(row + 1, k)
            A.swap_columns(row + 1, k)
            D.swap_rows(row + 1, k)

            b = A[row + 1, row]**(-1)
            A.add_multiple_of_column(row, row + 1, b**q)
            A.add_multiple_of_row(row, row + 1, b)
            D.add_multiple_of_row(row, row + 1, b)

        # Eliminate below-diagonal entries in the current column
        a = -A[row, row]**(-1)
        for i in range(row + 1, n):
            b = A[i, row] * a
            if not b.is_zero():
                A.add_multiple_of_column(i, row, b**q)
                A.add_multiple_of_row(i, row, b)
                D.add_multiple_of_row(i, row, b)

        if row == n - 1:
            break

    # Count how many variables have been used
    if row == n - 1:
        r = n if not A[n - 1, n - 1].is_zero() else n - 1

    # Normalize diagonal elements to 1
    for i in range(r):
        a = A[i, i]
        if not a.is_one():
            b = conj_square_root(a)
            D.rescale_row(i, 1 / b)

    return D.inverse()

In [3]:
q = 11
F = GF(q**2)
U=matrix(F,[[1,4,7],[4,1,4],[7,4,1]])

In [6]:
A = base_change_hermitian_gap_source(U); A*A.H == U

True

In [21]:
n = 6  # Number of vertices
p = 0.5  # Probability of an edge between any two vertices
G = graphs.RandomGNP(n, p)  # Create a random graph with n vertices and edge probability p
U = G.adjacency_matrix().change_ring(F)  # Get the adjacency matrix
A = base_change_hermitian_gap_source(U)
print(U)
print(A)

[0 0 1 1 1 1]
[0 0 0 0 1 0]
[1 0 0 0 1 0]
[1 0 0 0 1 1]
[1 1 1 1 0 1]
[1 0 0 1 1 0]
[     6*z2  7*z2 + 6         0         0         0         0]
[        0         0 6*z2 + 10  2*z2 + 1         0         0]
[     6*z2  4*z2 + 5         0         0         0         0]
[     6*z2  4*z2 + 5         0         0      6*z2  7*z2 + 6]
[       z2         0 10*z2 + 2         0         0         0]
[     6*z2  4*z2 + 5         0         0      6*z2  4*z2 + 5]


In [24]:
U.eigenvalues()

[10, 5, 5*z4^3 + 6*z4^2 + 3*z4, 9*z4^3 + 4*z4^2 + 4*z4, z4^3 + 3*z4^2 + 7*z4 + 2, 7*z4^3 + 9*z4^2 + 8*z4 + 5]

In [23]:
A.eigenvalues()

[z2 + 9, 6*z10^9 + 8*z10^8 + 9*z10^7 + 3*z10^6 + 9*z10^5 + 10*z10^4 + z10^3 + 4*z10^2 + 2*z10 + 1, 6*z10^9 + 7*z10^8 + 10*z10^7 + z10^6 + 6*z10^5 + 8*z10^4 + 10*z10^3 + 7*z10^2 + 3*z10 + 4, 4*z10^9 + 8*z10^8 + 9*z10^7 + 9*z10^6 + 5*z10^5 + 7*z10^4 + 3*z10^3 + z10 + 5, 7*z10^9 + 7*z10^8 + 3*z10^7 + 3*z10^6 + 3*z10^5 + 7*z10^3 + 9*z10^2 + 5*z10 + 9, 3*z10^9 + 3*z10^8 + 3*z10^7 + 7*z10^6 + z10^5 + 8*z10^4 + z10^3 + 10*z10^2 + 8*z10 + 9]